# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902086


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/89 [00:00<?, ?it/s]

Rendering models:   2%|▏         | 2/89 [00:06<04:42,  3.24s/it]

Rendering models:   3%|▎         | 3/89 [00:06<03:18,  2.31s/it]

Rendering models:   4%|▍         | 4/89 [00:06<02:21,  1.67s/it]

Rendering models:   6%|▌         | 5/89 [00:06<01:41,  1.21s/it]

...

Rendering models:  89%|████████▉ | 79/89 [00:13<00:00, 13.13it/s]

Rendering models:  91%|█████████ | 81/89 [00:13<00:00, 12.16it/s]

Rendering models:  93%|█████████▎| 83/89 [00:13<00:00,  9.95it/s]

Rendering models:  99%|█████████▉| 88/89 [00:13<00:00, 12.18it/s]

equidad1                               0.114109
kayleebug2017                          0.002239
yafeat                                 0.000985
not-logged-in-7fda7748d862237450e1     0.000297
not-logged-in-12e9cbecd83afb2188a8     0.000235
Drew_Farnsworth                        0.000543
Thedunkmasta                           0.001121
not-logged-in-6d073ad4cbbd00d3ccf3    12.693358
Thedunkmasta                           0.001208
47degreesnorth                         0.000261
not-logged-in-a2035c44f0ad873d19d7     0.021644
Alexander00                            0.001391
msross                                 0.002175
clizardo                               0.000107
clizardo                               0.000489
clizardo                               0.000384
mkirms                                 0.000420
msross                                 0.000450
mchaffin                               0.000220
djcameron                              0.000258
stgomes                                0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())